In [2]:
import numpy as np
import switching_times as st

%load_ext autoreload
%autoreload 2

In [3]:
wwtp = st.plant()

In [4]:
lambda_MELT = 0.00183*60
lambda_IDLE = 0.00085*60
mu_MELT = 66.92400 
mu_IDLE = 94.89100
k = 12400.
k_MELT = 250.
k_IDLE = 9.
dap = 1/1000000 * (10 + 6.8 * np.array([12.5,0.01,0.125,0.12,12.4,12.3,2.3,4,12.6,11,12.5,10,12.6,11,7.5,5,11.8,2.4,0.3,7.6,0.3,2,4,12.5]))
dap = np.concatenate((dap,dap))
#dap[-4] += 1000

In [5]:
x0 = np.array([24., 0.]) # IC for ODE
p_const = np.array([lambda_MELT, mu_MELT, lambda_IDLE, 
                    mu_IDLE, k, k_MELT, k_IDLE]) # constant parameters of system
p_dynamic = np.zeros(48 + 49) # day-ahead prices and day-ahead times -> vary in each new optimization -> therefore dynamic
for k in range(0, 48):
    p_dynamic[k] = dap[k]
for k in range(0, 49):
    p_dynamic[48 + k] = k * 60.
t0 = 0. # optimization start
tf = 24. * 60. # optimization end
dt = 0.1 # optimization discretization
n_s = 5 # number of switches
p_optimize = np.zeros(2 * n_s) # initial value of optimization variables
lower_bound = np.zeros(2 * n_s) # lower bounds for optimization variables
upper_bound = np.zeros(2 * n_s) # upper bounds for optimization variables
on_bound = np.array([0., 100000., 8.*60.])
off_bound = np.array([0., 1300000.])
for k in range(0, 2 * n_s):
    lower_bound[k], upper_bound[k] = t0, tf
p_optimize[n_s] = on_bound[0] + 1.
for k in range(1, n_s):
    p_optimize[k] = p_optimize[n_s + k - 1] + off_bound[0] + 1.
    p_optimize[n_s + k] = p_optimize[k] + on_bound[0] + 1.
    
    
#on_bound = np.array([6., tf])
#off_bound = np.array([20., tf])
p_optimize = np.array([  57.6,  309.6,  561.6,  813.6, 1065.6,  144. ,  396. ,  648. ,900. , 1152. ])
#p_optimize = np.concatenate((p_python[n_s:],p_python[:n_s]))
#tf

In [6]:
p_optimize

array([  57.6,  309.6,  561.6,  813.6, 1065.6,  144. ,  396. ,  648. ,
        900. , 1152. ])

In [7]:
on_times = p_optimize[n_s:] - p_optimize[:n_s]
off_times = np.append( p_optimize[1:n_s] - p_optimize[(n_s):(-1)], np.array(tf - p_optimize[-1]))
on_times, off_times            

(array([86.4, 86.4, 86.4, 86.4, 86.4]),
 array([165.6, 165.6, 165.6, 165.6, 288. ]))

In [8]:
wwtp.set_p_const(p_const)
wwtp.set_p_dynamic(p_dynamic)
wwtp.set_p_optimize(p_optimize)
wwtp.set_t0(t0)
wwtp.set_tf(tf)
wwtp.set_dt(dt)
wwtp.set_x0(x0)
wwtp.set_lower_bound(lower_bound)
wwtp.set_upper_bound(upper_bound)
wwtp.set_on_bound(on_bound)
wwtp.set_off_bound(off_bound)

In [9]:
print(wwtp.get_p_const())
print("-")
print(wwtp.get_p_dynamic())
print("-")
print(wwtp.get_p_optimize())
print("-")
print(wwtp.get_t0())
print("-")
print(wwtp.get_tf())
print("-")
print(wwtp.get_dt())
print("-")
print(wwtp.get_x0())
print("-")
print(wwtp.get_lower_bound())
print("-")
print(wwtp.get_upper_bound())
print("-")
print(wwtp.get_on_bound())
print("-")
print(wwtp.get_off_bound())
print("-")
print(wwtp.get_p_optimize())
print("-")
print(wwtp.get_p_dynamic())
print("-")

[1.0980e-01 6.6924e+01 5.1000e-02 9.4891e+01 1.2400e+04 2.5000e+02
 9.0000e+00]
-
[9.5000e-05 1.0068e-05 1.0850e-05 1.0816e-05 9.4320e-05 9.3640e-05
 2.5640e-05 3.7200e-05 9.5680e-05 8.4800e-05 9.5000e-05 7.8000e-05
 9.5680e-05 8.4800e-05 6.1000e-05 4.4000e-05 9.0240e-05 2.6320e-05
 1.2040e-05 6.1680e-05 1.2040e-05 2.3600e-05 3.7200e-05 9.5000e-05
 9.5000e-05 1.0068e-05 1.0850e-05 1.0816e-05 9.4320e-05 9.3640e-05
 2.5640e-05 3.7200e-05 9.5680e-05 8.4800e-05 9.5000e-05 7.8000e-05
 9.5680e-05 8.4800e-05 6.1000e-05 4.4000e-05 9.0240e-05 2.6320e-05
 1.2040e-05 6.1680e-05 1.2040e-05 2.3600e-05 3.7200e-05 9.5000e-05
 0.0000e+00 6.0000e+01 1.2000e+02 1.8000e+02 2.4000e+02 3.0000e+02
 3.6000e+02 4.2000e+02 4.8000e+02 5.4000e+02 6.0000e+02 6.6000e+02
 7.2000e+02 7.8000e+02 8.4000e+02 9.0000e+02 9.6000e+02 1.0200e+03
 1.0800e+03 1.1400e+03 1.2000e+03 1.2600e+03 1.3200e+03 1.3800e+03
 1.4400e+03 1.5000e+03 1.5600e+03 1.6200e+03 1.6800e+03 1.7400e+03
 1.8000e+03 1.8600e+03 1.9200e+03 1.9800e+03 2.

In [ ]:
wwtp.solve()

In [ ]:
np.array(wwtp.get_p_optimize_ipopt())

In [ ]:
print(wwtp.get_init_status())
print(wwtp.get_solve_status())

In [ ]:
p_opt[(n_s):(-1)]

In [ ]:
p_opt = wwtp.get_p_optimize_ipopt()
on_times =  p_opt[n_s:] - p_opt[:n_s]
off_times =   p_opt[1:n_s] - p_opt[(n_s):(-1)]    #np.append( p_opt[1:n_s] - p_opt[(n_s):(-1)], np.array(tf - p_opt[-1]))
on_times, off_times#, np.sum(on_times) + np.sum(off_times)

In [ ]:
print(wwtp.get_p_optimize_ipopt() - wwtp.get_p_optimize())

In [ ]:
wwtp.get_p_optimize_ipopt()

In [ ]:
wwtp.get_p_optimize()